# 6장 데이터 로딩과 저장, 파일 형식

In [121]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [124]:
!cat examples/ex1.csv

df = pd.read_csv("examples/ex1.csv")
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [128]:
pd.read_csv("examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [130]:
parsed = pd.read_csv("examples/csv_mindex.csv", index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [133]:
!cat examples/ex3.txt
result = pd.read_csv("examples/ex3.txt", sep="\s+") #s+ ->spacebar/가 복수
result

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\3239158813.py:2: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("examples/ex3.txt", sep="\s+") #s+ ->spacebar/가 복수


            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [4]:
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [139]:
result = pd.read_csv("examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [142]:
#pd.options.display.max_rows = 10
result = pd.read_csv("examples/ex6.csv")

result = pd.read_csv("examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [8]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64') #Series객체 생성
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0) #데이터프레임의 특정 컬럼 값 추출

tot = tot.sort_values(ascending=False)
tot
tot[:10] #Series 슬라이싱해서 추출

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용 #encoding 생략 가능
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [143]:
data = pd.read_csv("examples/ex5.csv")
data #dataframe에 데이터 저장
data1 = data[['a','b','d']] #인덱싱 : dataframe의 이후 데이터만 추출해서 csv로 저장
data1
data1.to_csv("examples/out.csv") #dataframe.to_csv(저장할 파일명)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


# 2. HTML 데이터
## 2.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


In [22]:
!pip install html5lib  lxml

#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [146]:
import pandas as pd
 
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
print(type(tables))
len(tables)

failures = tables[0] #인덱싱
print(type(failures))
failures.head()

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


,Bank NameBank,CityCity,StateSt,CertCert,Acquiring InstitutionAI,Closing DateClosing,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543
3,Signature Bank,New York,NY,57053,"Flagstar Bank, N.A.","March 12, 2023",10540
4,Silicon Valley Bank,Santa Clara,CA,24735,First–Citizens Bank & Trust Company,"March 10, 2023",10539


In [147]:
# 1) 컬럼명 수정하기
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

,BankNameBank,CityCity,StateSt,CertCert,AcquiringInstitutionAI,ClosingDate,FundFund
0,Citizens Bank,Sac City,IA,8758,Iowa Trust & Savings Bank,"November 3, 2023",10545
1,Heartland Tri-State Bank,Elkhart,KS,25851,"Dream First Bank, N.A.","July 28, 2023",10544
2,First Republic Bank,San Francisco,CA,59017,"JPMorgan Chase Bank, N.A.","May 1, 2023",10543


In [149]:
# 2) 데이터 변환 : 시간 /날짜 문자열 -> datetime객체 변환
close_timestamps = pd.to_datetime(failures["ClosingDate"])
close_timestamps.head()
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2023      5
2016      5
2020      4
2004      4
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [19]:
!pip install html5lib  lxml

   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   -------------- ------------------------ 41.0/112.2 kB 487.6 kB/s eta 0:00:01
   -------------------------------------- 112.2/112.2 kB 935.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
    --------------------------------------- 0.1/3.8 MB 1.1 MB/s eta 0:00:04
   - -------------------------------------- 0.1/3.8 MB 939.4 kB/s eta 0:00:04
   - -------------------------------------- 0.1/3.8 MB 901.1 kB/s eta 0:00:05
   - -------------------------------------- 0.2/3.8 MB 821.4 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/3.8 MB 787.7 kB/s eta 0:00:05
   -- ------------------------------------- 0.2/3.8 MB 765.3 kB/s eta 0:00:05
   --- ---------------

In [150]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
type(df_list)
len(df_list)
# 데이터프레임 확인
#for df in df_list:
    #print(df)
df_list[1]

C:\Users\Admin\python_basic_1\venv\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,83700.0,400.0,-0.48%,100.0,4996708.0,5969783.0,55.93,16963684.0,39.28,4.15,NaN
2,2.0,SK하이닉스,187400.0,1000.0,-0.53%,5000.0,1364276.0,728002.0,54.90,3041976.0,-14.97,-15.61,NaN
3,3.0,LG에너지솔루션,371500.0,8500.0,-2.24%,500.0,869310.0,234000.0,4.87,171737.0,70.27,6.36,NaN
4,4.0,삼성전자우,69100.0,100.0,-0.14%,100.0,568615.0,822887.0,75.61,1000412.0,32.43,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,49.0,포스코인터내셔널,46450.0,1050.0,-2.21%,5000.0,81716.0,175923.0,6.96,443044.0,12.12,13.36,NaN
77,50.0,에코프로머티,114700.0,4800.0,-4.02%,500.0,79126.0,68985.0,2.48,289195.0,1349.41,0.91,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
# 데이터프레임 탐색
df = df_list[1]
# 결측치 처리-종목명
# 조건색인을 이용해서 boolean 인덱싱으로 새로운 데이터프레임 생성
#1. 조건색인
cond = df['종목명'].notnull() # notna()
#2. 조건색인을 데이터프레임에 넣어주기 -True 값만 추출
data = df[cond] # 조건색인
data
data.head()
# 최종결과를 데이터프레임 형태로 저장 - df.to_csv(파일명)
data.to_csv("Naver_kospi.csv")

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
1,1.0,삼성전자,83700.0,400.0,-0.48%,100.0,4996708.0,5969783.0,55.93,16963684.0,39.28,4.15,NaN
2,2.0,SK하이닉스,187400.0,1000.0,-0.53%,5000.0,1364276.0,728002.0,54.90,3041976.0,-14.97,-15.61,NaN
3,3.0,LG에너지솔루션,371500.0,8500.0,-2.24%,500.0,869310.0,234000.0,4.87,171737.0,70.27,6.36,NaN
4,4.0,삼성전자우,69100.0,100.0,-0.14%,100.0,568615.0,822887.0,75.61,1000412.0,32.43,NaN,NaN
5,5.0,삼성바이오로직스,790000.0,5000.0,-0.63%,2500.0,562275.0,71174.0,11.99,41085.0,65.55,9.12,NaN


## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 

In [154]:
# 1. requests 모듈 설치하기
#!pip install requests
# 2. requests 모듈 임포트하기
import requests
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
print(resp.status_code)
#4. 웹페이지 내용 텍스트 확인
resp.text
#원하는 데이터 처리 방법
# 5. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
#print(type(data))
#len(data)
#6. json 내용 확인
data[0]
#7. json의 딕셔너리
data[0].keys() # 키 정보 출력
#8. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                       "labels", "state"])
issues.head()
#데이터프레임을 저장
issues.to_csv("pandas_issues.csv")

200


dict_keys(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'draft', 'pull_request', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason'])

### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [42]:
import requests
# url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC"
# response = requests.get(url)
params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}



response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

200
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC




<!DOCTYPE html>
<html lang="ko">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <base href="/home" />
    <meta name="robots" content="noindex,nofollow"/>
    <meta name="referrer" content="always"/>
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="https://section.blog.naver.com/favicon.ico?3"/>
    




 

	
		<meta property="og:title" content="파이썬 : 네이버 블로그"/>

		
			
			
			<meta property="og:image" content="https://blogimgs.pstatic.net/nblog/mylog/post/og_default_image_160610.png"/>
			
		
        <meta property="og:description" content="당신의 모든 기록을 담는 공간"/>

		
	
	

<meta property="me:feed:serviceId" content="blog" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>네이버 블로그</title>

    <link rel="stylesheet" type="text/css" href="htt

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [44]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    #웹페이지의 원하는 데이터로 정보 추출
    #1. html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')
    
    #2. 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())

    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    for title in titles:
        print(title.get_text()) #제목 텍스트 가져오기
else : 
    print(response.status_code)

비주얼스튜디오코드 파이썬 질문합니다,
비주얼스튜디오코드 파이썬 질문합니다,
맥북 파이썬 idel
파이썬 모듈 미리 불러오기
파이썬과 자바
IDLE 파이썬 단축키 알려주세요
파이썬강좌 독학할때 뭐보나요??
파이썬이 미래에 도움이 될까요?
파이썬강의 듣는데가 있나요?
파이썬 라이선스 분석
파이썬독학할 때 어떻게 공부를... 


## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제] 크롬드라이버를 이용하여 타겟 웹사이트 실행하기

In [48]:
!pip install selenium

In [52]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome()
# 타겟 웹페이지 열기
url = "https://www.naver.com"
driver.get(url)

### [문제] Selnium을 이용하여 타겟 웹사이트 실행하기/검색어의 제목과 URL 출력
* 크롬 브라우저를 이용하여 "open ai" 사용
* 구글 개발 도구를 이용

In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

#1. Chrome 브라우저 실행
driver = webdriver.Chrome()

#2. 타겟 웹페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(5)

try:
    #1. 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    #2. 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    #3. Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    #4. 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g") #<div class = "g"></div>

    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
    # 브라우저 창 닫기
    driver.quit()


제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://namu.wiki/w/OpenAI

제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI

제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI

제목: OpenAI
URL: https://www.youtube.com/@OpenAI

제목: OpenAI
URL: https://twitter.com/OpenAI

제목: 설명
URL: https://www.google.com/search?q=OpenAI&sca_esv=954f74c53a241618&source=hp&ei=R9gUZtKvJYOhvr0Pv7OE-Ak&iflsig=ANes7DEAAAAAZhTmV0z39V7ZE23AqhpUqhhmlRn0MQUh&udm=&ved=0ahUKEwiS76qJubSFAxWDkK8BHb8ZAZ8Q4dUDCA8&uact=5&oq=OpenAI&gs_lp=Egdnd3Mtd2l6IgZPcGVuQUlILlAAWBJwAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#



### [응용문제]: 시가총액이 높은 상위 500개 기업의 데이터를 가져오기
* 1~50 페이지 중 10개 페이지만 데이터 읽어오기
* 10개 페이지를 하나의 데이터로 합치기
* URL의 페이지 번호 실행하기
* 저장하기

In [ ]:
from selenium import webdriver
import pandas as pd
import time

In [60]:
#Chrome 브라우저를 열고 웹 페이지로 이동
driver = webdriver.Chrome()

#시가총액 상위 500개 기업 데이터를 가져올 URL
url = 'https://finance.naver.com/sise/sise_market_sum.naver?&page='

#10개 페이지의 데이터를 저장할 리스트
kospi500=[]

#10개 페이지에 대해 반복하여 데이터 수집
for num in range(1,11): #페이지 번호 제어
    full_url=url+str(num)
    driver.get(full_url) #페이지로 이동
    time.sleep(2) #페이지가 로드될 때까지 기다림
    html = driver.page_source #페이지 HTML 가져오기
    data = pd.read_html(html) #HTML을 데이터프레임으로 읽기 시도
    df=data[1] #두 번째 테이블에 종목 데이터가 있음
    df=df[df['종목명'].notnull()] #조건 색인으로 결측치 처리
    kospi500.append(df) #선택된 테이블을 리스트에 추가
    
#10개 페이지의 데이터를 하나의 데이터프레임으로 합치기
combined_df = pd.concat(kospi500)

#결과 출력
combined_df.to_csv("combined_kospi.csv")

#드라이버 닫기
driver.quit()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\2030683725.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(html) #HTML을 데이터프레임으로 읽기 시도
C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\2030683725.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(html) #HTML을 데이터프레임으로 읽기 시도
C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\2030683725.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(html) #HTML을 데이터프레임으로 읽기 시도
C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\2030683725.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and wi

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00007FF6A4C070C2+63090]
	(No symbol) [0x00007FF6A4B72D12]
	(No symbol) [0x00007FF6A4A0EC65]
	(No symbol) [0x00007FF6A49ECA7C]
	(No symbol) [0x00007FF6A4A7D687]
	(No symbol) [0x00007FF6A4A92AC1]
	(No symbol) [0x00007FF6A4A76D83]
	(No symbol) [0x00007FF6A4A483A8]
	(No symbol) [0x00007FF6A4A49441]
	GetHandleVerifier [0x00007FF6A500262D+4238301]
	GetHandleVerifier [0x00007FF6A503F78D+4488509]
	GetHandleVerifier [0x00007FF6A5037A6F+4456479]
	GetHandleVerifier [0x00007FF6A4CE0606+953270]
	(No symbol) [0x00007FF6A4B7E5DF]
	(No symbol) [0x00007FF6A4B792B4]
	(No symbol) [0x00007FF6A4B793EB]
	(No symbol) [0x00007FF6A4B69C24]
	BaseThreadInitThunk [0x00007FF922DA257D+29]
	RtlUserThreadStart [0x00007FF92420AA58+40]


# [문제] 삼성전자의 시가총액의 일별 시세 데이터 가져오기
* Selnium을 사용하여 웹 페이지를 열고, 페이지의 HTML을 가져와서 BeautifulSoup
* 이후 데이터를 데이터프레임으로 변환하여 출력하기

In [81]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [116]:
# Chrome 웹 드라이버 실행
driver = webdriver .Chrome()

In [117]:
time.sleep(2)

In [118]:
# target 웹페이지 url - 사전작업
# 삼성전자 일별 시세 정보 페이지
samsung_url = "https://finance.naver.com/item/sise_day.naver?code=005930&page="
samsung_daily_kospi=[]

In [120]:
for num in range(1,50):
    full_url = samsung_url + str(num)
    # 페이지 요청
    driver.get(full_url)
    
    try:
        #페이지의 HTML 가져오기
        html = driver.page_source
        df = pd.read_html(html)
        #결측치 처리
        cond = df[0]['날짜'].notnull()
        samsung_day = df[0][cond]
        samsung_daily_kospi.append(samsung_day)
        print(samsung_daily_kospi)
    except Exception as e:
        print("Network Error",e)

    # finally:
        # driver.quit()

samsung_daily_kospi_all = pd.concat(samsung_daily_kospi)
samsung_daily_kospi_all.to_csv("examples/saumsung_daily_kospi_2024_2020.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\876365209.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)


[            날짜       종가     전일비       시가       고가       저가         거래량
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2024.03.27  79800.0   100.0  79200.0  80000.0  79200.0  17424595.0,             날짜       종가     전일비       시가       고가       저가         거래량
1   2024.03.26  79900.0  1700.0  79700.0  80100.0  79200.0  30551494.0
2   2024.03.25  78200.0   700.0  79600.0  79800.0  77800.0  18703996.0
3   

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\876365209.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)


[            날짜       종가     전일비       시가       고가       저가         거래량
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2024.03.27  79800.0   100.0  79200.0  80000.0  79200.0  17424595.0,             날짜       종가     전일비       시가       고가       저가         거래량
1   2024.03.26  79900.0  1700.0  79700.0  80100.0  79200.0  30551494.0
2   2024.03.25  78200.0   700.0  79600.0  79800.0  77800.0  18703996.0
3   

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\876365209.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)


...

[            날짜       종가     전일비       시가       고가       저가         거래량
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2024.03.27  79800.0   100.0  79200.0  80000.0  79200.0  17424595.0,             날짜       종가     전일비       시가       고가       저가         거래량
1   2024.03.26  79900.0  1700.0  79700.0  80100.0  79200.0  30551494.0
2   2024.03.25  78200.0   700.0  79600.0  79800.0  77800.0  18703996.0
3   

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\876365209.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)


[            날짜       종가     전일비       시가       고가       저가         거래량
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2024.03.27  79800.0   100.0  79200.0  80000.0  79200.0  17424595.0,             날짜       종가     전일비       시가       고가       저가         거래량
1   2024.03.26  79900.0  1700.0  79700.0  80100.0  79200.0  30551494.0
2   2024.03.25  78200.0   700.0  79600.0  79800.0  77800.0  18703996.0
3   

C:\Users\Admin\AppData\Local\Temp\ipykernel_18796\876365209.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)


[            날짜       종가     전일비       시가       고가       저가         거래량
1   2024.04.09  83600.0   900.0  84500.0  84900.0  83100.0  18342168.0
2   2024.04.08  84500.0     0.0  85200.0  86000.0  84500.0  18953232.0
3   2024.04.05  84500.0   800.0  84500.0  85000.0  83800.0  18883752.0
4   2024.04.04  85300.0  1200.0  85200.0  85500.0  84300.0  25248934.0
5   2024.04.03  84100.0   900.0  84300.0  85000.0  83500.0  30493347.0
9   2024.04.02  85000.0  3000.0  82900.0  85000.0  82900.0  37077944.0
10  2024.04.01  82000.0   400.0  83200.0  83300.0  82000.0  20116513.0
11  2024.03.29  82400.0  1600.0  81200.0  82500.0  80900.0  27126366.0
12  2024.03.28  80800.0  1000.0  79400.0  81000.0  79200.0  25084812.0
13  2024.03.27  79800.0   100.0  79200.0  80000.0  79200.0  17424595.0,             날짜       종가     전일비       시가       고가       저가         거래량
1   2024.03.26  79900.0  1700.0  79700.0  80100.0  79200.0  30551494.0
2   2024.03.25  78200.0   700.0  79600.0  79800.0  77800.0  18703996.0
3   

# [응용문제] : 시가총액이 높은 상위 500개 기업의 데이터를 가져오기
* 1~50 페이지 중 10개의 페이지만 데이터 가져오기
* 10개의 페이지를 하나의 데이터로 합치기
* URL의 페이지 번호 실행하기